In [37]:
# Config:
# Feature - real, imaginary, magnitude
# Sample - 100 CIRs
# Non-zero coefficients - 10
# -------------------------------------------------------------------------------------------------------
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs']

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

# Extract data for ALICE and BOB channels
alice_CIRs = data_cir[:100, alice_channel, :, :]  # Shape: (9797, 251, 2)
eve_CIRs = data_cir[:100, eve_channel, :, :]  # Shape: (9797, 251, 2)

# ALICE features - real, imaginary, magnitude
alice_real = alice_CIRs[:, :, 0]
alice_imag = alice_CIRs[:, :, 1]
alice_magnitude = np.abs(alice_real + 1j * alice_imag)
alice_features = np.hstack((alice_real, alice_imag, alice_magnitude))

# EVE features - real, imaginary, magnitude
eve_real = eve_CIRs[:, :, 0]
eve_imag = eve_CIRs[:, :, 1]
eve_magnitude = np.abs(eve_real + 1j * eve_imag)
eve_features = np.hstack((eve_real, eve_imag, eve_magnitude))

# Atoms for our dictionary - Reshape Alice's data to flatten each sample.
alice_atoms = alice_features.reshape(100, -1)  # Each sample becomes a long vector.
eve_atoms = eve_features.reshape(100, -1)

# Create labels for Alice and Eve.
alice_labels = np.zeros(alice_atoms.shape[0])  # Label '0' for Alice.
eve_labels = np.ones(eve_atoms.shape[0])       # Label '1' for Eve.

# Combine data and labels.
X = np.vstack((alice_atoms, eve_atoms))  # Combined data.
y = np.hstack((alice_labels, eve_labels))  # Combined labels.

# Build the dictionary using the training data.
dictionary = X.T  # Transpose so that each column is an atom.

# Now, we will classify each test sample and keep track of predictions.
predictions = []

In [38]:

test_real = eve_CIRs[0, :, 0]
test_imag = eve_CIRs[0, :, 1]
test_magnitude = np.abs(test_real + 1j * test_imag)
test_signal = np.hstack((test_real, test_imag, test_magnitude))

# Create the OMP model.
omp = OrthogonalMatchingPursuit(n_nonzero_coefs=4, tol=1e-1)

# Fit the model to find the sparse coefficients.
omp.fit(dictionary, test_signal)
coefficients = omp.coef_

# Initialize residuals list for this test signal.
residuals = []
coef_class = np.zeros_like(coefficients)

# We have two classes: Alice (0) and Eve (1).
for i in range(2):
    if i == 0:
        # For Alice, keep coefficients corresponding to Alice's atoms.
        coef_class[:100] = coefficients[:100]
        # print(coef_class.shape)
    else:
        # For Eve, keep coefficients corresponding to Eve's atoms.
        coef_class[100:] = coefficients[100:]
    
    # # Reconstruct the signal using only the coefficients from one class.
    reconstructed_signal = dictionary @ coef_class
    
    # # Calculate the residual.
    residual = np.linalg.norm(test_signal - reconstructed_signal)
    # # Add the residual to the list.
    residuals.append(residual)

# Predict the class with the smallest residual.
predicted_class = np.argmin(residuals)
predictions.append(predicted_class)

print(predictions)
print(residuals)

# accuracy = accuracy_score(y, predictions)
# print(f"\nClassification Accuracy: {accuracy * 100:.2f}%")

[1]
[67730.49184820674, 1.1975901719559808e-10]
